# 🚀 Colab GPU Daemon - Solution 2: Hybrid Local Dev + Remote Exec

**Objectif**: Exécuter automatiquement des expériences GPU depuis commits GitHub

## 📋 Workflow:
1. **Développer localement** dans VSCode avec Copilot
2. **Commit + Push** vers branche `gpu-experiments`
3. **Ce notebook détecte** le nouveau commit
4. **Exécute expériences** définies dans `experiments.json`
5. **Push résultats** automatiquement

## ✅ Avantages:
- ✨ Développement 100% local (VSCode + Copilot)
- 🔒 Zéro SSH fragile
- ♻️ Async naturel
- 📦 Tout dans Git (reproductible)
- 🎯 Background execution sans interruption

---

**⚠️ Important**: Ce notebook doit tourner en continu sur Colab Pro pour surveiller le repo.

## 🔧 Étape 1: Vérification GPU

In [ ]:
# Vérifier GPU disponible
!nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv,noheader

# Test PyTorch
import torch
print(f"\n{'='*60}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"{'='*60}\n")

## 📦 Étape 2: Mount Google Drive (Persistance)

In [ ]:
# Mount Google Drive pour persister résultats
from google.colab import drive
drive.mount('/content/drive')

# Créer dossier outputs dans Drive
!mkdir -p /content/drive/MyDrive/panini_colab_outputs
!mkdir -p /content/outputs

# Symlink vers Drive
!rm -f /content/outputs 2>/dev/null || true
!ln -s /content/drive/MyDrive/panini_colab_outputs /content/outputs

print("✅ Google Drive monté et lié à /content/outputs")

## 🔑 Étape 3: Configuration Git (pour push résultats)

In [ ]:
# Configuration Git (remplacer par vos infos)
!git config --global user.name "Colab GPU Daemon"
!git config --global user.email "colab@panini-gpu.local"

print("✅ Git configuré pour commits automatiques")

## 📥 Étape 4: Clone Repository

In [ ]:
# Clone repo Panini (branche gpu-experiments)
import os
from pathlib import Path

REPO_URL = "https://github.com/stephanedenis/Panini.git"
BRANCH = "gpu-experiments"
WORK_DIR = Path("/content/work")

# Remove existing si présent
if WORK_DIR.exists():
    !rm -rf {str(WORK_DIR)}

# Clone
!git clone -b {BRANCH} {REPO_URL} {str(WORK_DIR)}

print(f"✅ Repo cloné dans {WORK_DIR}")

## 📚 Étape 5: Installer Dépendances

In [ ]:
# Installer requirements.txt si présent
%cd /content/work

req_file = Path("requirements.txt")
if req_file.exists():
    print("📦 Installation des dépendances...")
    !pip install -r requirements.txt -q
    print("✅ Dépendances installées")
else:
    print("⚠️  Pas de requirements.txt trouvé")

## 🤖 Étape 6: Lancer le Daemon

**⚠️ Important**: Cette cellule tourne indéfiniment! Elle surveille le repo et exécute les expériences.

**Pour arrêter**: Menu Runtime → Interrupt execution (ou `Ctrl+M I`)

In [ ]:
# Lancer le daemon Python
%cd /content/work

# Le script daemon est dans tools/colab_daemon_setup.py
!python tools/colab_daemon_setup.py

---

## 📊 Monitoring (Cellules optionnelles)

In [ ]:
# Voir le log du daemon
!tail -50 /content/daemon.log

In [ ]:
# Voir experiments.json actuel
import json
from pathlib import Path

exp_file = Path("/content/work/experiments.json")
if exp_file.exists():
    with open(exp_file) as f:
        experiments = json.load(f)
    
    print(f"{'='*60}")
    print(f"EXPERIMENTS STATUS")
    print(f"{'='*60}\n")
    
    for exp in experiments:
        name = exp.get('name', 'unknown')
        status = exp.get('status', 'unknown')
        duration = exp.get('duration', 0)
        
        icon = {'completed': '✅', 'failed': '❌', 'pending': '⏳', 
                'timeout': '⏱️', 'error': '💥'}.get(status, '❓')
        
        print(f"{icon} {name}")
        print(f"   Status: {status}")
        if duration:
            print(f"   Duration: {duration:.1f}s")
        print()
else:
    print("⚠️  Aucun fichier experiments.json")

In [ ]:
# Voir les outputs générés
!ls -lh /content/outputs/

In [ ]:
# Monitoring GPU en temps réel
import time

print("🔍 GPU Monitoring (Ctrl+C pour arrêter)\n")

try:
    while True:
        !nvidia-smi --query-gpu=utilization.gpu,utilization.memory,memory.used,memory.total,temperature.gpu --format=csv,noheader
        time.sleep(5)
except KeyboardInterrupt:
    print("\n✋ Monitoring arrêté")